In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import joblib
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn import metrics


2022-07-31 19:47:10.953447: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 19:47:10.953495: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('DataNew.csv',index_col=0)

In [3]:
#First Step -> Tokenize and padd
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>") #num_words : the maximum number of words to keep, based on word frequency (for the 45232 products information), #oov : out-of-vocabulary token, it is agregated to words which are not in the 10000 most frequent ones
tokenizer.fit_on_texts(df['Text']) # encoding words of information to integers, Updates internal vocabulary based on a list of sequences.
word_index = tokenizer.word_index
print("Amount of different words through the 45232 products information:",len(word_index))
sequences = tokenizer.texts_to_sequences(df['Text'])
print("Amount of sequences:", len(sequences)) #Should be equal to the amount of products 
print("Sequences Examples:",sequences[0], sequences[1], sequences[2], sequences[3], sequences[4]) #This is how the first 5 products info are represented, each number represent a word in "word_index" dict.
padded = pad_sequences(sequences, padding="post") #This takes the maxlen parameter as a referene and makes all the lists of the same lenght adding zeros on the left spaces, if nothins is pass ot will take the longest sequence as a reference
print("First Sequence padded: \n",padded[0])
print(len(padded[0]), len(padded[1]), len(padded[2])) #Should all be equal to 
print("Dataset Padded Shape:", padded.shape)

Amount of different words through the 45232 products information: 20558
Amount of sequences: 51007
Sequences Examples: [3081, 1660, 27, 61, 3, 16, 115, 51, 1660, 163, 4490, 10, 2669, 26, 61] [3081, 1336, 64, 1, 27, 61, 742, 1589, 440, 4490, 10, 2669, 26, 1796, 941, 169, 148, 158, 5276] [3081, 1336, 27, 61, 3, 16, 115, 51, 1336, 163, 4490, 10, 2669, 26, 61] [1363, 1161, 27, 1336, 61, 61, 1336, 2271, 27, 27, 6290, 23] [3081, 349, 27, 61, 3, 16, 115, 51, 349, 163, 4490, 10, 2669, 26, 61]
First Sequence padded: 
 [3081 1660   27   61    3   16  115   51 1660  163 4490   10 2669   26
   61    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
59 59 59
Dataset Padded Shape: (51007, 59)


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
# Splitting dataset in train/test set
X_train, X_test, y_train, y_test = train_test_split(padded, df[df.columns[1:-1]], 
                                                    test_size=0.3, 
                                                    random_state=0)

In [6]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model

In [7]:
num_classes = y_train.shape[1]
print(num_classes)
max_words = len(word_index) + 1
print(max_words)
maxlen = len(padded[0])
print(maxlen)

852
20559
59


In [8]:
from keras.layers import Activation, Flatten, Conv1D

In [17]:
optimizer = tf.keras.optimizers.Adam()

model = Sequential()
model.add(Embedding(max_words, 300, input_length=maxlen))
model.add(Dropout(0.21))
model.add(Conv1D(300, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D()) # This layer creates a convolution kernel that is convolved with the layer input over a single spatial (or temporal) dimension to produce a tensor of outputs.
model.add(Dropout(0.21))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Precision','Recall', tf.keras.metrics.AUC()])

callbacks = [
    ReduceLROnPlateau(),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 59, 300)           6167700   
                                                                 
 dropout_2 (Dropout)         (None, 59, 300)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 57, 300)           270300    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 300)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_3 (Dropout)         (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 852)               256452    
                                                      

In [10]:
plot_model(model, to_file='model_nn_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [18]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.3,
                    callbacks=callbacks)

Epoch 1/50
781/781 [==============================] - 55s 69ms/step - loss: 0.0255 - precision: 0.2511 - recall: 0.1811 - auc_1: 0.8677 - val_loss: 0.0091 - val_precision: 0.8808 - val_recall: 0.4640 - val_auc_1: 0.9472 - lr: 0.0010
Epoch 2/50
781/781 [==============================] - 58s 74ms/step - loss: 0.0073 - precision: 0.8954 - recall: 0.5495 - auc_1: 0.9579 - val_loss: 0.0053 - val_precision: 0.9149 - val_recall: 0.6806 - val_auc_1: 0.9756 - lr: 0.0010
Epoch 3/50
781/781 [==============================] - 49s 63ms/step - loss: 0.0047 - precision: 0.9141 - recall: 0.7019 - auc_1: 0.9785 - val_loss: 0.0039 - val_precision: 0.9232 - val_recall: 0.7700 - val_auc_1: 0.9833 - lr: 0.0010
Epoch 4/50
781/781 [==============================] - 49s 62ms/step - loss: 0.0035 - precision: 0.9286 - recall: 0.7790 - auc_1: 0.9864 - val_loss: 0.0034 - val_precision: 0.9221 - val_recall: 0.8166 - val_auc_1: 0.9866 - lr: 0.0010
Epoch 5/50
781/781 [==============================] - 49s 63ms/step 

In [13]:
def get_standar_metrics(predictions, y_test):
  a = tranform_binary(predictions)
  print("Evaluation Metrics")
  print('-'*30)
  print("F1 Score:",metrics.f1_score(y_test, a, average="micro"))
  print("Recall Score:",metrics.recall_score(y_test, a, average="micro"))
  print("Precision Score:",metrics.precision_score(y_test, a, average="micro"))
  print("Hamming Loss:",metrics.hamming_loss(y_test, a), "\n")

In [14]:
def tranform_binary(predictions):
    a = predictions.tolist()
    for i in range(len(a)):
        for j in range(len(a[i])):
            if a[i][j] >= 0.5:
                a[i][j] = 1
            else:
                a[i][j] = 0

    a = np.array(a)
    return a

In [16]:
#Model Evaluation with Standar Metrics
predictions = model.predict(X_test, batch_size=32)
get_standar_metrics(predictions, y_test)

479/479 [==============================] - 2s 4ms/step
Evaluation Metrics
------------------------------
F1 Score: 0.9058894960534305
Recall Score: 0.8814301030566921
Precision Score: 0.9317451139123395
Hamming Loss: 0.000641961946152508 



In [56]:
modeloF =  keras.models.load_model('model-conv1d.h5')

In [63]:
inp = 'metra radio dash multikit select gm vehicle black expand online assortment compatible select gm vehicle plastic material'

x = tokenizer.texts_to_sequences([inp])

x = pad_sequences(x)

pre = modeloF.predict(x)

1/1 [==============================] - 0s 85ms/step


In [62]:

joblib.dump(tokenizer,'tokenizer.plk')

['tokenizer.plk']

In [64]:
predScores = [score for pred in pre for score in pred]

predDict = {}
for cla,score in zip(df.columns[1:],predScores):
        predDict[cla] = score

In [65]:
import operator
sorted(predDict.items(), key=operator.itemgetter(1),reverse=True)[:5]

[('Car Electronics & GPS', 0.9999984),
 ('Car Audio Installation Parts', 0.9999419),
 ('Car Installation Parts & Accessories', 0.99992317),
 ('Back-Up & Dash Cameras', 0.02803315),
 ('Smartphone & iPod Car Connectors', 0.021082692)]

In [133]:
def ModelPrediction(text):

    tokenizer = joblib.load('tokenizer.plk')

    sequence = tokenizer.texts_to_sequences([text])

    sequence = pad_sequences(x,maxlen=59)

    model = keras.models.load_model('model-conv1d.h5')

    prediction = model.predict(sequence)
    
    predScores = [score for pred in prediction for score in pred]

    predDict = {}

    for cla,score in zip(df.columns[1:],predScores):
        
        predDict[cla] = score
    
    preSort = sorted(predDict.items(), key=operator.itemgetter(1),reverse=True)[:3]


    result = []
    for i in preSort:
        
        if i[1] > 0.80:
            
            result.append(i[0])

    return result

In [134]:
ModelPrediction(inp)
    

1/1 [==============================] - 0s 44ms/step


['Car Electronics & GPS',
 'Car Audio Installation Parts',
 'Car Installation Parts & Accessories']